In [1]:
# read the validation logs from test_comprehensive and perform some data analysis
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import scipy

test_filepath = '../logs/morning-sea-7/test.csv'
df = pd.read_csv(test_filepath)
df

,Unnamed: 0,path,study_id,gt,pred,conf,ent,attn
0,0,D:\Datasets\TMED2\approved_users_only\view_and...,3532s1,2,2,0.707254,0.807508,1.309366
1,1,D:\Datasets\TMED2\approved_users_only\view_and...,3532s1,2,2,0.965417,0.152781,1.807265
2,2,D:\Datasets\TMED2\approved_users_only\view_and...,3532s1,2,2,0.954975,0.195102,1.175923
3,3,D:\Datasets\TMED2\approved_users_only\view_and...,3532s1,2,2,0.989899,0.057539,2.037830
4,4,D:\Datasets\TMED2\approved_users_only\view_and...,3532s1,2,2,0.911655,0.359928,2.363879
...,...,...,...,...,...,...,...,...
1305,1305,D:\Datasets\TMED2\approved_users_only\view_and...,5396s1,1,2,0.924427,0.320200,2.571994
1306,1306,D:\Datasets\TMED2\approved_users_only\view_and...,5396s1,1,2,0.972177,0.143443,2.343115
1307,1307,D:\Datasets\TMED2\approved_users_only\view_and...,5396s1,1,2,0.913771,0.347794,2.264605
1308,1308,D:\Datasets\TMED2\approved_users_only\view_and...,5396s1,1,2,0.751035,0.731597,2.301659


In [2]:
def softmax_normalization(arr): # expects length-N array, not matrix
    return scipy.special.softmax(arr)

def aggregate_by_confidence(num_classes, pred, confidence, attention):
    total = np.zeros(num_classes)
    for i in range(len(pred)):
        weight = confidence[i] * attention[i]
        total[pred[i]] += weight
    return total

In [3]:
# calculate the overall accuracy and f1
gt = []
pred = []
study_ids = df['study_id'].unique()
num_classes = len(df['gt'].unique())
for ids in study_ids:
    study = df[df['study_id']==ids]
    attn_norm = softmax_normalization(study['attn'].to_numpy())
    # if attention values are all equal, then vote via confidence value
    # if attention values are not equal, vote via attention and confidence value
    final_prediction = aggregate_by_confidence(num_classes, study['pred'].to_numpy(), study['conf'].to_numpy(), attn_norm)
    final_prediction_norm = softmax_normalization(final_prediction)
    #print(final_prediction_norm)
    gt.append(study.iloc[0]['gt'])
    pred.append(np.argmax(final_prediction_norm))
print(classification_report(gt, pred))

              precision    recall  f1-score   support

           0       0.48      0.64      0.55        25
           1       0.00      0.00      0.00        33
           2       0.61      0.85      0.71        60

    accuracy                           0.57       118
   macro avg       0.36      0.50      0.42       118
weighted avg       0.41      0.57      0.48       118



In [4]:
print(confusion_matrix(gt, pred))

[[16  1  8]
 [ 8  0 25]
 [ 9  0 51]]


In [5]:
# # check the correlation between correct-view attention and other-view attention
# attn_right = df[df['view_label'].isin(['PLAX', 'PSAX'])]
# attn_wrong = df[~df['view_label'].isin(['PLAX', 'PSAX'])]
# scipy.stats.pearsonr(attn_right['attn'].to_numpy(), attn_wrong['attn'].to_numpy())